<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/0928_DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
1. 웹이서 불러온 데이터로 df 생성
2. pickle 저장
'''

In [ ]:
'''
1. pickle 로드
2. 로드된 df에서 마지막 수집날짜 확인
3. (마지막 수집날짜+1 ~ 오늘) 데이터 수집 (웹에서 불러옴)
4. 기존 df와 3.의 df 병합 (concat)
5. 새로운 df 다시 pickle 저장
'''

In [ ]:
import json
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime
import pickle
from google.colab import drive

drive.mount('/content/drive')
res_path = '/content/drive/MyDrive/k-project/output/'
# pickle 경로를 설정하기 위해

Mounted at /content/drive


In [ ]:
headers = {'User-Agent': 'Chrome/93.0.4577.63'}

data = {
    'mktsel': 'ALL',
    'typeNo': '0',
    'searchText':'',
    'bld': 'dbms/comm/finder/finder_stkisu'
}

url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

r = requests.post(url, data, headers = headers)
j = json.loads(r.text)

df = pd.json_normalize(j['block1'])
print(len(df))
# print(df.info())
# print(df.iloc[1]['full_code'])
df

2584


,full_code,short_code,codeName,marketCode,marketName,marketEngName,ord1,ord2
0,KR7060310000,060310,3S,KSQ,코스닥,KOSDAQ,,16
1,KR7095570008,095570,AJ네트웍스,STK,유가증권,KOSPI,,16
2,KR7006840003,006840,AK홀딩스,STK,유가증권,KOSPI,,16
3,KR7054620000,054620,APS홀딩스,KSQ,코스닥,KOSDAQ,,16
4,KR7265520007,265520,AP시스템,KSQ,코스닥,KOSDAQ,,16
...,...,...,...,...,...,...,...,...
2579,KR7000542001,000547,흥국화재2우B,STK,유가증권,KOSPI,,16
2580,KR7000541003,000545,흥국화재우,STK,유가증권,KOSPI,,16
2581,KR7003280005,003280,흥아해운,STK,유가증권,KOSPI,,16
2582,KR7037440005,037440,희림,KSQ,코스닥,KOSDAQ,,16


In [ ]:
# 한국시간 설정
import pytz

KST = pytz.timezone('Asia/Seoul')
today = int(datetime.now(KST).strftime("%Y%m%d"))
print(today)

20210928


In [ ]:
df = df.set_index('short_code')
full_code = df.loc['005930']['full_code']
# full_code = 'KR7005930003'

In [ ]:
# 실행 안함
# 웹에서 불러오는게 아니라 pickle로 저장한 데이터를 사용하기 위해


strtDd = '19000101'
endDd = today

data2 = { 'bld': 'dbms/MDC/STAT/issue/MDCSTAT23902',
  'isuCd': full_code,
  'isuCd2': '',
  'strtDd': strtDd,
  'endDd': endDd,
  'share': '1',
  'money': '1',
  'csvxls_isNo': 'false',}

url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

r2 = requests.post(url2, data2, headers)
# print(r2)
j2 = json.loads(r2.text)
df2 = pd.DataFrame(j2['output'])
df2

In [ ]:
# 데이터프레임 전체 열 출력
pd.set_option('display.max_columns', None)

## Save pickle
with open(res_path + "df2.pickle","wb") as fw:
    pickle.dump(df2, fw)

In [ ]:
## Load pickle
# pickle로 저장되었던 데이터 출력 (어제까지의 데이터)
with open(res_path + "df2.pickle","rb") as fr:
    df2 = pickle.load(fr)
print(df2)

          TRD_DD  ISU_CD  ...         ACC_TRDVAL               MKTCAP
0     2021/09/24  005930  ...    947,054,144,957  461,464,191,115,000
1     2021/09/23  005930  ...  1,318,209,157,168  462,061,169,370,000
2     2021/09/17  005930  ...  1,250,677,861,383  460,867,212,860,000
3     2021/09/16  005930  ...  1,000,180,664,750  454,300,452,055,000
4     2021/09/15  005930  ...    987,846,688,544  459,673,256,350,000
...          ...     ...  ...                ...                  ...
6683  1995/05/08  005930  ...     11,127,115,000    6,578,606,045,000
6684  1995/05/06  005930  ...      5,810,535,000    6,632,974,690,000
6685  1995/05/04  005930  ...     21,609,210,000    6,660,159,012,500
6686  1995/05/03  005930  ...     47,649,710,000    6,714,527,657,500
6687  1995/05/02  005930  ...     16,676,735,000    6,497,053,077,500

[6688 rows x 15 columns]


In [ ]:
# 매일 pickle 데이터 불러와서 읽은 후 0행, 0열의 날짜 확인
# 초기 데이터가 없는 데이터? -> if문 or try~except 예외처리
import re

date = df2.iloc[0,0]
yesterday = int(re.sub('/','',date))
print(yesterday)

20210924


In [ ]:
strtDd = yesterday+1 #20210925
endDd = today #20210928


data2 = { 'bld': 'dbms/MDC/STAT/issue/MDCSTAT23902',
  'isuCd': full_code,
  'isuCd2': '',
  'strtDd': strtDd,
  'endDd': endDd,
  'share': '1',
  'money': '1',
  'csvxls_isNo': 'false',}

url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

r2 = requests.post(url2, data2, headers)
# print(r2)
j2 = json.loads(r2.text)
df3 = pd.DataFrame(j2['output'])
df3

,TRD_DD,ISU_CD,ISU_NM,MKT_NM,SECUGRP_NM,TDD_CLSPRC,FLUC_TP_CD,CMPPRVDD_PRC,FLUC_RT,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL,MKTCAP
0,2021/09/28,005930,삼성전자,KOSPI,주권,"76,900",2,-800,-1.03,"77,700","77,800","76,700","6,822,878","526,210,928,900","459,076,278,095,000"
1,2021/09/27,005930,삼성전자,KOSPI,주권,"77,700",1,400,0.52,"77,300","77,700","77,000","11,699,798","906,520,162,508","463,852,104,135,000"


In [ ]:
result = pd.concat([df3, df2], ignore_index = True)
print(result)

          TRD_DD  ISU_CD  ...         ACC_TRDVAL               MKTCAP
0     2021/09/28  005930  ...    526,210,928,900  459,076,278,095,000
1     2021/09/27  005930  ...    906,520,162,508  463,852,104,135,000
2     2021/09/24  005930  ...    947,054,144,957  461,464,191,115,000
3     2021/09/23  005930  ...  1,318,209,157,168  462,061,169,370,000
4     2021/09/17  005930  ...  1,250,677,861,383  460,867,212,860,000
...          ...     ...  ...                ...                  ...
6685  1995/05/08  005930  ...     11,127,115,000    6,578,606,045,000
6686  1995/05/06  005930  ...      5,810,535,000    6,632,974,690,000
6687  1995/05/04  005930  ...     21,609,210,000    6,660,159,012,500
6688  1995/05/03  005930  ...     47,649,710,000    6,714,527,657,500
6689  1995/05/02  005930  ...     16,676,735,000    6,497,053,077,500

[6690 rows x 15 columns]


In [ ]:
## Save pickle (업데이트된 데이터프레임 다시 저장)
with open(res_path + "df2.pickle","wb") as fw:
    pickle.dump(result, fw)

In [ ]:
# schedule 라이브러리: 정해진 시간에 파이썬(Python) 스크립트 자동 실행
# 다른 종목 수집?
# 최종 산출물에 필요한 컬럼?
# 파일 저장 형태 : json ?

In [1]:
!git clone https://github.com/baeksumin/k-project.git

Cloning into 'k-project'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [3]:
!git branch

fatal: not a git repository (or any of the parent directories): .git
